# [ATL14](https://nsidc.org/data/atl14/)


## Downloading data using earthaccess

In [ ]:
import earthaccess

auth = earthaccess.login()
earthaccess.__version__

In [ ]:
temporal_range = ("2021-01", "2023-01")
max_files = 10

download_list = []

results = earthaccess.search_data(short_name="ATL14",
                                  cloud_hosted=True,
                                  temporal = temporal_range,
                                  count=max_files)
for granule in results:
    if granule.size() < 500:
        download_list.append(granule)
        
print(f"total granules listed under 500MB: {len(download_list)}")
files = earthaccess.download(download_list, local_path="../data/ATL14")

## Loading data

In [ ]:
import xarray as xr
import hvplot.xarray
from cartopy import crs
import pathlib
import geoviews as gv
import geoviews.feature as gf

import pyproj
from shapely.geometry import box
from shapely.ops import transform

gv.extension('bokeh', 'matplotlib')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Listing the files in /data/ATL14
files = []
for file in pathlib.Path('../data/ATL14').glob('*.nc'):
    files.append(str(file))
files

## Exploring ATL16 with xarray

* A) Opening a single granule
> xarray uses h5py or netcdf4 to open `.h5` and `.nc` files, it opens the file and based on the signature it selects the backend
> If the files are local this should be pretty straight-forward. 

* B) Plotting the granule using [hvplot](https://hvplot.holoviz.org/user_guide/Gridded_Data.html)

In [ ]:
# Single granule, xarray opens hdf5 files using the netcdf4/h5py libraries.
ds = xr.open_dataset(files[0])
ds

### Exploring data geolocation using GeoViews

We create a bounding box using the dataset coordinate values (x,y)  

In [ ]:
project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:3413'), # source coordinate system
    pyproj.Proj(init='epsg:4326'))


bounds = transform(project.transform, box(ds.x.min(), ds.y.min(), ds.x.max(), ds.y.max()))
bounds

Now we visualize where the bouding box is using GeoViews. Since we already know the data is in Polar Nort projection we use it and set the projected bounds as the limit to our map.

In [ ]:
projected_bounds = {
    "lat": (-3314693.24, 3314693.24),
    "lon": ( -3314693.24, 3314693.24)
}

(gf.coastline.geoms('110m').opts(projection=crs.NorthPolarStereo(),
                                 xlim=projected_bounds["lon"], 
                                 ylim=projected_bounds["lat"],
                                 global_extent=False,
                                 width=600, height=400,
                                 color='black') 
* gv.Polygons(bounds).opts(global_extent=False,
                           color='green') 
* gf.ocean)

ATL14 is an elevation dataset, we can plot a variable with hvplot via its xarray accessor, each variable in a gridded dataset can be plotted using `.hvplot()`

In [ ]:
variable = "h"
ds[variable].hvplot(
    'x', 'y',
    crs=crs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70),
    projection=crs.NorthPolarStereo(),
    frame_height=540,
    frame_width=800,
    cmap='viridis',
    project=False,
    rasterize=True,
    coastline=True
)

## Validating 

In [ ]:
## TODO